In [19]:
import matplotlib
import torch
from torchcontrib.optim import SWA

In [20]:
## import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
#matplotlib.use("Agg")
from matplotlib import figure  # pylint: disable=g-import-not-at-top
from matplotlib.backends import backend_agg
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
try:
  import seaborn as sns  # pylint: disable=g-import-not-at-top
  HAS_SEABORN = True
except ImportError:
  HAS_SEABORN = False

tfd = tfp.distributions

In [21]:
#Model
model = torch.nn()

TypeError: 'module' object is not callable

In [ ]:
base_opt = torch.optim.SGD(model.parameters(), lr=0.1)
opt = torchcontrib.optim.SWA(base_opt, swa_start=10, swa_freq=5, swa_lr=0.05)
for _ in range(100):
    opt.zero_grad()
    loss_fn(model(input), target).backward()
    opt.step()
opt.swap_swa_sgd()

In [14]:
def make_mixture_posterior_fn(mixture_components):
    """
    Creates the mixture of Gaussians posterior distribution.
    """
    if mixture_components == 1:
    # See the module docstring for why we don't learn the parameters here.
    # default_mvn_fn returns a standard non-trainable MVN
        return tfp.layers.default_multivariate_normal_fn
    
    def _random_posterior_fn(dtype, shape, name, trainable, add_variable_fn):
    # signature matches tfp.layers.default_multivariate_normal_fn
#    loc = add_variable_fn(
#       name=name + "/loc", shape=[mixture_components,] + list(shape),  trainable=trainable)
#    raw_scale_diag = add_variable_fn(
#       name=name+"/scale", shape=[mixture_components,] + list(shape), trainable=trainable)
    loc = add_variable_fn(
        name=name + "/loc", shape=list(shape) + [mixture_components,],  trainable=trainable)
    raw_scale_diag = add_variable_fn(
        name=name+"/scale", shape=list(shape) + [mixture_components,], trainable=trainable)
    mixture_logits = add_variable_fn(
        name=name+"/logit", shape=[mixture_components], trainable=trainable)

    return tfd.MixtureSameFamily(
        components_distribution=tfd.MultivariateNormalDiag(
            loc=loc,
            scale_diag=tf.nn.softplus(raw_scale_diag)),
        mixture_distribution=tfd.Categorical(logits=mixture_logits),
        name="posterior")
  return _random_posterior_fn